<!-- Copyright (c) 2021-2024, InterDigital Communications, Inc
All rights reserved.

Redistribution and use in source and binary forms, with or without 
modification, are permitted (subject to the limitations in the disclaimer 
below) provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, 
this list of conditions and the following disclaimer.
* Redistributions in binary form must reproduce the above copyright notice, 
this list of conditions and the following disclaimer in the documentation 
and/or other materials provided with the distribution.
* Neither the name of InterDigital Communications, Inc nor the names of its 
contributors may be used to endorse or promote products derived from this 
software without specific prior written permission.

NO EXPRESS OR IMPLIED LICENSES TO ANY PARTY'S PATENT RIGHTS ARE GRANTED BY 
THIS LICENSE. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND 
CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT 
NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A 
PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER 
OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, 
EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, 
PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; 
OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, 
WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR 
OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF 
ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. -->

# Knowledge Distillation Applied to Auto-Encoder

In [ ]:
import io
import math
import numpy as np

from PIL import Image

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader

from compressai.datasets import Vimeo90kDataset

import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

from models import TeacherAE, StudentAE

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

## Visualise Models

In [ ]:
# %%script false --no-raise-error
from torchsummary import summary

# model = TeacherAE()
model = StudentAE()
summary(model, input_size=(3, 256, 256))

In [ ]:
%%script false --no-raise-error
import graphviz
from torchview import draw_graph
graphviz.set_jupyter_format("png")
from models import TeacherAE, StudentAE

# model = TeacherAE()
model = StudentAE()
model_graph = draw_graph(model, input_size=(1, 3, 256, 256), mode="eval", device="meta")
model_graph.visual_graph

In [ ]:
%%script false --no-raise-error
import torch
from torchviz import make_dot
from models import TeacherAE, StudentAE

# model = TeacherAE()
model = StudentAE()
x = torch.randn(1, 3, 256, 256)
latent, y = model(x)

make_dot(y.mean(), params=dict(model.named_parameters()))
# make_dot(y.mean(), params=dict(model.named_parameters()), show_attrs=True, show_saved=True)

## Load Models

In [ ]:
# Teacher Model
# id = 243807 # Denoising
# id = 254462 # Reconstruction
id = 274518 # Reconstruction
teacher_net = TeacherAE()
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
teacher_net.load_state_dict(checkpoint["state_dict"])
teacher_net = teacher_net.eval().to(device)

print(f"Parameters: {sum(p.numel() for p in teacher_net.parameters())}")

In [ ]:
# Student Model
# id = 244247 # Denoising
# id = 254463 # Reconstruction
id = 274520 # Reconstruction
student_net = StudentAE()
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_net.load_state_dict(checkpoint["state_dict"])
student_net = student_net.eval().to(device)

print(f"Parameters: {sum(p.numel() for p in student_net.parameters())}")

In [ ]:
# Student Model with KD
# id = 244336 # Denoising
# id = 254597 # Reconstruction
id = 298339 # Reconstruction
student_kd_net = StudentAE()
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_kd_net.load_state_dict(checkpoint["state_dict"])
student_kd_net = student_kd_net.eval().to(device)

print(f"Parameters: {sum(p.numel() for p in student_kd_net.parameters())}")

## Models Comparison

### Saint Malo

In [ ]:
test_transforms = transforms.Compose(
    [transforms.RandomCrop((256,256)), transforms.ToTensor()]
)

img = Image.open("../data/assets/stmalo_fracape.png").convert("RGB")
x = test_transforms(img).unsqueeze(0).to(device)
print(x.shape)

In [ ]:
with torch.no_grad():
    _, teacher_out = teacher_net.forward(x)
    _, student_out = student_net.forward(x)
    _, student_kd_out = student_kd_net.forward(x)

criterion = nn.MSELoss()
teacher_loss = criterion(teacher_out, x)
student_loss = criterion(student_out, x)
student_kd_loss = criterion(student_kd_out, x)

teacher_diff = torch.mean((teacher_out - x).abs(), axis=1).squeeze().cpu()
student_diff = torch.mean((student_out - x).abs(), axis=1).squeeze().cpu()
student_kd_diff = torch.mean((student_kd_out - x).abs(), axis=1).squeeze().cpu()

teacher_out = transforms.ToPILImage()(teacher_out.squeeze().cpu())
student_out = transforms.ToPILImage()(student_out.squeeze().cpu())
student_kd_out = transforms.ToPILImage()(student_kd_out.squeeze().cpu())

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(16, 12))

orig = transforms.ToPILImage()(x.squeeze().cpu())
outs = [teacher_out, student_out, student_kd_out]
diffs = [teacher_diff, student_diff, student_kd_diff]
losses = [teacher_loss, student_loss, student_kd_loss]

for ax in axs.ravel():
    ax.axis("off")

for i in range(3):
    axs[i, 0].imshow(orig)
    axs[i, 0].title.set_text("Original")

    axs[i, 1].imshow(outs[i])
    axs[i, 1].title.set_text("Reconstructed")

    axs[i, 2].imshow(diffs[i], cmap="viridis")
    axs[i, 2].title.set_text(f"Difference ({losses[i]:.3f})")

plt.show()